In [1]:
import spacy
from spacy import displacy
from spacy.tokens import DocBin
from pathlib import Path
from tqdm.auto import tqdm


/home/abrolhus/programs/anaconda3/envs/nlp/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Leitura do dataset

### Primeiro é feito a leitura do dataset, dividido aleatoriamente em train (80%), test (10%), eval (10%)

In [46]:
def get_docs(path):
    text_labels = []
    texts = []
    labels = []
    docs = []

    i = 0;
    for folder in Path(path).iterdir():
        print(folder.name)
        for file in folder.glob('*.txt'):
            #print(file)
            with file.open() as f:
                text_labels.append(folder.name)
                texts.append(f.read())
            i += 1
            labels.append(folder.name)
            data = tuple(zip(texts, text_labels))        
    nlp = spacy.load("pt_core_news_md")
   
    
    for doc, label in tqdm(nlp.pipe(data, as_tuples=True), total = len(data)): 
        # tqdm ==> barra de progresso
        doc.cats[labels[0]] = 0
        doc.cats[labels[1]] = 0
        doc.cats[labels[2]]  = 0
        doc.cats[labels[3]]  = 0
        
        if (label== labels[0]):
            doc.cats[labels[0]] = 1
        elif (label==labels[1]):
            doc.cats[labels[1]] = 1
        elif (label==labels[2]):
            doc.cats[labels[2]]  = 1
        else:
            doc.cats[labels[3]]  = 1
        #print(doc.cats)
        docs.append(doc)
    return docs, data

In [3]:
docs_train1, data_train1 = get_docs("dataset_train/")

3_Ensino_Medio
['3_Ensino_Medio']
2_Ensino_Fundamental_II
['3_Ensino_Medio', '2_Ensino_Fundamental_II']
4_Ensino_Superior
['3_Ensino_Medio', '2_Ensino_Fundamental_II', '4_Ensino_Superior']
.DS_Store
1_Ensino_Fundamental_I
['3_Ensino_Medio', '2_Ensino_Fundamental_II', '4_Ensino_Superior', '1_Ensino_Fundamental_I']


100%|█████████████████████████████████████████████████████████████████████████████████████| 1691/1691 [02:18<00:00, 12.18it/s]


In [4]:
docs_test1, data_test1 = get_docs("dataset_test/")

3_Ensino_Medio
['3_Ensino_Medio']
2_Ensino_Fundamental_II
['3_Ensino_Medio', '2_Ensino_Fundamental_II']
4_Ensino_Superior
['3_Ensino_Medio', '2_Ensino_Fundamental_II', '4_Ensino_Superior']
1_Ensino_Fundamental_I
['3_Ensino_Medio', '2_Ensino_Fundamental_II', '4_Ensino_Superior', '1_Ensino_Fundamental_I']


100%|███████████████████████████████████████████████████████████████████████████████████████| 206/206 [00:15<00:00, 13.38it/s]


In [5]:
docs_eval1, data_eval1 = get_docs("dataset_eval/")

3_Ensino_Medio
['3_Ensino_Medio']
2_Ensino_Fundamental_II
['3_Ensino_Medio', '2_Ensino_Fundamental_II']
4_Ensino_Superior
['3_Ensino_Medio', '2_Ensino_Fundamental_II', '4_Ensino_Superior']
1_Ensino_Fundamental_I
['3_Ensino_Medio', '2_Ensino_Fundamental_II', '4_Ensino_Superior', '1_Ensino_Fundamental_I']


100%|███████████████████████████████████████████████████████████████████████████████████████| 206/206 [00:16<00:00, 12.12it/s]


## Criação dos Modelos


### Modelo 1

In [6]:
doc_bin = DocBin(docs=docs_train1)
doc_bin.to_disk("./doc_bins/textcat_1_train.spacy")

doc_bin_eval = DocBin(docs=docs_eval1)
doc_bin_eval.to_disk("./doc_bins/textcat_1_eval.spacy")

#### Treinando modelo 1

In [7]:
!python -m spacy train textcat_config.cfg --verbose --output ./textcat_output --paths.train ./doc_bins/textcat_1_train.spacy --paths.dev ./doc_bins/textcat_1_eval.spacy

[2022-10-10 07:31:45,549] [DEBUG] Config overrides from CLI: ['paths.train', 'paths.dev']
ℹ Saving to output directory: textcat_output
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2022-10-10 07:31:45,767] [INFO] Set up nlp object from config
[2022-10-10 07:31:45,788] [DEBUG] Loading corpus from path: doc_bins/textcat_1_eval.spacy
[2022-10-10 07:31:45,791] [DEBUG] Loading corpus from path: doc_bins/textcat_1_train.spacy
[2022-10-10 07:31:45,792] [INFO] Pipeline: ['textcat']
[2022-10-10 07:31:45,796] [INFO] Created vocabulary
[2022-10-10 07:31:45,799] [INFO] Finished initializing nlp object
[2022-10-10 07:32:16,650] [INFO] Initialized pipeline components: ['textcat']
✔ Initialized pipeline

============================= Training pipeline =============================
[2022-10-10 07:32:16,668] [DEBUG] Loading corpus from path: doc_bins/textcat_1_eval.spacy
[2022-10-10 07:32:16,670] [DEBUG] Loading corpus from path: doc_bins/textcat_1_train.sp

#### Testando manualmente modelo 1

In [ ]:
nlp_textcat = spacy.load("textcat_output/model-best")
with open('./dataset_test/3_Ensino_Medio/123_950.txt') as f:
    test_text = f.read()
    doc2 = nlp_textcat(test_text)
    print("Texto: "+ test_text)
    print("Nivel:"+ " 3_Ensino_Medio")
    print("Nivel do modelo:") 
    print(doc2.cats)

### Modelo 2

com pré-processamento dos textos, removendo *stop words*, pontuação e deixando somente palavras base.

In [47]:
def get_clean_token_list(doc):
    clean_token_list = []
    for token in doc:
        if not token.is_punct and not token.is_stop:
            txt = token.lemma_.lower().strip()
            if txt != "":
                clean_token_list.append(txt)
    return clean_token_list

In [48]:
docs_train2, data_train2 = get_docs("dataset_train/")
docs_test2,  data_test2  = get_docs("dataset_test/")
docs_eval2,  data_eval2  = get_docs("dataset_eval/")
print(docs_test2[0].cats)

3_Ensino_Medio
2_Ensino_Fundamental_II
4_Ensino_Superior
1_Ensino_Fundamental_I


100%|█████████████████████████████████████████████████████████████████████████████████████| 1691/1691 [02:12<00:00, 12.74it/s]


3_Ensino_Medio
2_Ensino_Fundamental_II
4_Ensino_Superior
1_Ensino_Fundamental_I


100%|███████████████████████████████████████████████████████████████████████████████████████| 206/206 [00:18<00:00, 11.13it/s]


3_Ensino_Medio
2_Ensino_Fundamental_II
4_Ensino_Superior
1_Ensino_Fundamental_I


100%|███████████████████████████████████████████████████████████████████████████████████████| 206/206 [00:16<00:00, 12.57it/s]


{'3_Ensino_Medio': 1}


In [52]:
clean_train =[(" ".join(get_clean_token_list(doc)), doc.cats) for doc in docs_train2]
clean_test = [(" ".join(get_clean_token_list(doc)), doc.cats) for doc in docs_test2]
clean_eval = [(" ".join(get_clean_token_list(doc)), doc.cats) for doc in docs_eval2]

In [53]:
print(clean_train[:10])
nlp = spacy.load("pt_core_news_sm")

[('inca passar chamar depreciativamente chiriguano preferer referir se mesmo guarani avá século xvi chegada colonizador europeu povo tupis temiminó tupiniquim tabajara aliar português potiguar caetés tamoio aliar francês carijós guarani aliar espanhol resultado destruição aldeia indígeno escravização doença trazir europeu fuga população indígeno interior continente aldeamento criar padre jesuíta redução missão índio ser catequizar trabalhar padre jesuíta perd cultura indígena importante fator estimular escravização índio mulher europeu chegar américa ataque aldeia indígeno apresamento índia tornar se saída obtenção mulher filho de esse união chamar mameluco tomar expedição escravizadora índio direção interior continente chamado entrada bandeira índio tupis reagir política escravização fugir interior continente atacar povoação português brasil importante rebelião tupi português confederação tamoios reunir tupinambá cabo frio atual brasileiro rio janeiro bertioga atual brasileiro paulo 1

In [41]:
clean_train_docs = []
clean_test_docs = []
clean_eval_docs = []

for item in clean_train:
    doc = nlp(item[0])
    doc.cats = item[1]
    clean_train_docs.append(doc)
for item in clean_test:
    print(item[1])
    doc = nlp(item[0])
    doc.cats = item[1]
    clean_test_docs.append(doc)
for item in clean_eval:
    doc = nlp(item[0])
    doc.cats = item[1]
    clean_eval_docs.append(doc)

{'3_Ensino_Medio': 0, '2_Ensino_Fundamental_II': 0, '4_Ensino_Superior': 0, '1_Ensino_Fundamental_I': 0}
{'3_Ensino_Medio': 0, '2_Ensino_Fundamental_II': 0, '4_Ensino_Superior': 0, '1_Ensino_Fundamental_I': 0}
{'3_Ensino_Medio': 0, '2_Ensino_Fundamental_II': 0, '4_Ensino_Superior': 0, '1_Ensino_Fundamental_I': 0}
{'3_Ensino_Medio': 0, '2_Ensino_Fundamental_II': 0, '4_Ensino_Superior': 0, '1_Ensino_Fundamental_I': 0}
{'3_Ensino_Medio': 0, '2_Ensino_Fundamental_II': 0, '4_Ensino_Superior': 0, '1_Ensino_Fundamental_I': 0}
{'3_Ensino_Medio': 0, '2_Ensino_Fundamental_II': 0, '4_Ensino_Superior': 0, '1_Ensino_Fundamental_I': 0}
{'3_Ensino_Medio': 0, '2_Ensino_Fundamental_II': 0, '4_Ensino_Superior': 0, '1_Ensino_Fundamental_I': 0}
{'3_Ensino_Medio': 0, '2_Ensino_Fundamental_II': 0, '4_Ensino_Superior': 0, '1_Ensino_Fundamental_I': 0}
{'3_Ensino_Medio': 0, '2_Ensino_Fundamental_II': 0, '4_Ensino_Superior': 0, '1_Ensino_Fundamental_I': 0}
{'3_Ensino_Medio': 0, '2_Ensino_Fundamental_II': 0, '4_

In [54]:
doc_bin = DocBin(docs=clean_train_docs)
doc_bin.to_disk("./doc_bins/textcat_2_train.spacy")

doc_bin_eval = DocBin(docs=clean_eval_docs)
doc_bin_eval.to_disk("./doc_bins/textcat_2_eval.spacy")

#### Treinando Modelo 2

In [55]:
!python -m spacy train textcat_config.cfg --verbose --output ./textcat_2_output --paths.train ./doc_bins/textcat_2_train.spacy --paths.dev ./doc_bins/textcat_2_eval.spacy

[2022-10-10 08:34:17,739] [DEBUG] Config overrides from CLI: ['paths.train', 'paths.dev']
✔ Created output directory: textcat_2_output
ℹ Saving to output directory: textcat_2_output
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2022-10-10 08:34:17,946] [INFO] Set up nlp object from config
[2022-10-10 08:34:17,968] [DEBUG] Loading corpus from path: doc_bins/textcat_2_eval.spacy
[2022-10-10 08:34:17,971] [DEBUG] Loading corpus from path: doc_bins/textcat_2_train.spacy
[2022-10-10 08:34:17,971] [INFO] Pipeline: ['textcat']
[2022-10-10 08:34:17,976] [INFO] Created vocabulary
[2022-10-10 08:34:17,979] [INFO] Finished initializing nlp object
[2022-10-10 08:34:27,546] [INFO] Initialized pipeline components: ['textcat']
✔ Initialized pipeline

============================= Training pipeline =============================
[2022-10-10 08:34:27,564] [DEBUG] Loading corpus from path: doc_bins/textcat_2_eval.spacy
[2022-10-10 08:34:27,566] [DEBUG] Loadin

#### Testando manualmente Modelo 2

In [57]:
nlp_textcat_2 = spacy.load("textcat_output/model-best")
print("Nivel certo: ", clean_test_docs[1].cats) 
clean_doc = nlp_textcat_2(clean_test_docs[1].text)
print("Texto: "+ clean_doc.text)
print("Nivel do modelo:") 
print(doc2.cats)

Nivel certo:  {'3_Ensino_Medio': 0, '2_Ensino_Fundamental_II': 0, '4_Ensino_Superior': 0, '1_Ensino_Fundamental_I': 0}
Texto: placa formar diversos tipo proteína desmoplaquina placoglobina placofilino dentre proteína conhecer caderina desmogleína desmocolino presente junção aderente proteína presente junção oclusivo nectino afadina formar complexo afadina-nectino ocorrer junção oclusivo desmossoma junção celular constituir parte de ele membrana célula membrana célula vizinho desmossoma consistir placa circular proteína placa desmossoma célula placa desmossoma partir filamento constituír proteína atravessar membrana plasmático atingir espaço intercelular associar filamento oriunr placa desmossoma célula vizinho associação filamento espaço intercelular manter firmemente unir placa desmossômico conseqüentemente célula contêr placa desmossoma voltar interior célula liga-se filamento queratina citoesqueleto promover firme ancoramento desmossomo estrutura celular poder desmossomas permitir a